In [38]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *

In [39]:
key_path = key_path
project = project_id
dataset = 'bronze'
table = 'bronze_ticker_data'
table_conca = f'{project}.{dataset}.{table}'

schema = [
    {'name': 'Date', 'type': 'DATE'},
    {'name': 'Ticker', 'type': 'STRING'},
    {'name': 'Adj Close', 'type': 'INTEGER'},
    {'name': 'Close', 'type': 'INTEGER'},
    {'name': 'High', 'type': 'INTEGER'},
    {'name': 'Low', 'type': 'INTEGER'},
    {'name': 'Open', 'type': 'INTEGER'},
    {'name': 'Volume', 'type': 'INTEGER'},
]

In [40]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [41]:
# Obtenemos todos los tickers
tickers_dow = si.tickers_dow()
tickers_sp500 = si.tickers_sp500()
tickers_nasdaq = si.tickers_nasdaq()

In [64]:
# Combinar todos los tickers en una sola lista
combined_tickers = tickers_dow + tickers_sp500 + tickers_nasdaq

# Usar un diccionario para eliminar duplicados pero mantener el primer encuentro
unique_tickers = list(dict.fromkeys(combined_tickers))

# max_date_by_ticker = list(set(dow + sp + nasdaq)) 
max_date_by_ticker = pd.DataFrame(unique_tickers, columns=['ticker'])
max_date_by_ticker['initial_date'] = datetime.strptime('2015-01-01', '%Y-%m-%d')

# Convertir todas las fechas a tz-naive (sin zona horaria)
max_date_by_ticker['initial_date'] = max_date_by_ticker['initial_date'].dt.tz_localize(None)

print(max_date_by_ticker)

      ticker initial_date
0       AAPL   2015-01-01
1       AMGN   2015-01-01
2       AMZN   2015-01-01
3        AXP   2015-01-01
4         BA   2015-01-01
...      ...          ...
4818  ZXYZ.A   2015-01-01
4819   ZXZZT   2015-01-01
4820    ZYME   2015-01-01
4821    ZYXI   2015-01-01
4822     ZZZ   2015-01-01

[4823 rows x 2 columns]


In [65]:
# Obtener la fecha maxima por ticker
try:
    current_data = bigquery.run_query(
        f"""
        SELECT
            * 
        FROM {table_conca} 
        """
    )
    # Convertir todas las fechas a tz-naive (sin zona horaria)
    current_data['date'] = current_data['date'].dt.tz_localize(None)

    # Sacamos la fecha máxima por ticker
    query_ticker = current_data.groupby('ticker')['date'].max().reset_index()
    query_ticker['date'] = pd.to_datetime(query_ticker['date'])

    #Mergeamos
    max_date_by_ticker = pd.merge(max_date_by_ticker, query_ticker, how='left', on='ticker')
    max_date_by_ticker['date'] = max_date_by_ticker[['date', 'initial_date']].max(axis=1)

    # Eliminar la columna auxiliar 'initial_date'
    max_date_by_ticker.drop(columns=['initial_date'], inplace=True)
    print(max_date_by_ticker)
    
except Exception as e:
    max_date_by_ticker
    print(max_date_by_ticker)

      ticker       date
0       AAPL 2024-08-08
1       AMGN 2024-08-08
2       AMZN 2024-08-08
3        AXP 2024-08-08
4         BA 2024-08-08
...      ...        ...
4818  ZXYZ.A 2015-01-01
4819   ZXZZT 2015-01-01
4820    ZYME 2015-01-01
4821    ZYXI 2015-01-01
4822     ZZZ 2015-01-01

[4823 rows x 2 columns]


In [44]:
# Para cada ticker, usa la fecha correspondiente
data = []

for _, row in max_date_by_ticker.iterrows():
    ticker = row['ticker']
    start_date = row['date'] + pd.Timedelta(days=1) 
    if isinstance(start_date, pd.Timestamp):
        start_date = start_date.to_pydatetime()  # Convertir a datetime de Python

    try:
        # Obtener datos históricos
        data_row = si.get_data(ticker, start_date=start_date, end_date=date.today(), index_as_date=False, interval="1d")
        # Agregar el símbolo como una columna en los datos históricos
        data_row['ticker'] = ticker
        # Añadir los datos al diccionario
        data.append(data_row)
        print(f"Datos obtenidos para {ticker}.")
    except Exception as e:
        print(f"Error al obtener datos para {ticker}: {e}")

Datos obtenidos para AAPL.
Datos obtenidos para AMGN.
Datos obtenidos para AMZN.
Datos obtenidos para AXP.
Datos obtenidos para BA.
Datos obtenidos para CAT.
Datos obtenidos para CRM.
Datos obtenidos para CSCO.
Datos obtenidos para CVX.
Datos obtenidos para DIS.
Datos obtenidos para DOW.
Datos obtenidos para GS.
Datos obtenidos para HD.
Datos obtenidos para HON.
Datos obtenidos para IBM.
Datos obtenidos para INTC.
Datos obtenidos para JNJ.
Datos obtenidos para JPM.
Datos obtenidos para KO.
Datos obtenidos para MCD.
Datos obtenidos para MMM.
Datos obtenidos para MRK.
Datos obtenidos para MSFT.
Datos obtenidos para NKE.
Datos obtenidos para PG.
Datos obtenidos para TRV.
Datos obtenidos para UNH.
Datos obtenidos para V.
Datos obtenidos para VZ.
Datos obtenidos para WMT.
Error al obtener datos para -: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
Error al obtener datos para --: {'chart': {'result': None, 'error': {'code'

In [76]:
# Concatenamos todos las listas obtenidas y convertimos en un dataframe
df = pd.concat(data, ignore_index=True)
df = pd.DataFrame(df)
df.head()

,date,open,high,low,close,adjclose,volume,ticker
0,2024-08-09,212.100006,216.779999,211.970001,216.240005,215.990005,42201600.0,AAPL
1,2024-08-12,216.070007,219.509995,215.600006,217.529999,217.529999,38028100.0,AAPL
2,2024-08-13,219.009995,221.889999,219.009995,221.270004,221.270004,44095400.0,AAPL
3,2024-08-09,323.750000,325.040009,319.329987,322.489990,322.489990,1786400.0,AMGN
4,2024-08-12,323.899994,323.899994,317.929993,320.000000,320.000000,1828700.0,AMGN


In [78]:
try:
    max_date_by_ticker['date'] = pd.to_datetime(max_date_by_ticker['date'], utc=True)
    df['date'] = pd.to_datetime(df['date'], utc=True)

    # Realizar el LEFT JOIN
    left_join = pd.merge(df, max_date_by_ticker, on=['ticker', 'date'], how='left')

    # Realizar el INNER JOIN
    inner_join = pd.merge(df, max_date_by_ticker, on=['ticker', 'date'], how='inner')

    # Identificar las filas del LEFT JOIN que no están en el INNER JOIN
    # Usar las columnas 'ticker' y 'date' para realizar la exclusión
    left_join_tuples = left_join[['ticker', 'date']].apply(tuple, axis=1)
    inner_join_tuples = inner_join[['ticker', 'date']].apply(tuple, axis=1)

    # Obtener los datos que están en left_join pero no en inner_join
    result = left_join[~left_join_tuples.isin(inner_join_tuples)]

    # Guardamos en Bigquery
    bigquery.save_dataframe(result, project, dataset, table, if_exists='append', schema=schema)

except Exception as e:
    print(e)

'id'
No se obtuvieron datos históricos.
